In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
df = pd.read_csv('insurance.csv')
df.head()

In [ ]:
# Data cleaning
df.isna().sum()

In [ ]:
# Transform Categorical data into numerical
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()

# Transform the 'sex' column
df['sex'] = label_encoder.fit_transform(df['sex'])

# Transform the 'smoker' column
df['smoker'] = label_encoder.fit_transform(df['smoker'])

# Transform the 'region' column
df['region'] = label_encoder.fit_transform(df['region'])
print(df.head())

In [ ]:
# Use 80% of the available data to train the model, the other 20% will be used to evaluate it
train_df = df.sample(frac=0.8,random_state=0)
test_df = df.drop(train_df.index)

train_labels = train_df.pop('expenses')
test_labels = test_df.pop('expenses')

In [ ]:
# Normalize data and create the model

normalizer = layers.Normalization(axis=-1)
normalizer.adapt(train_df.to_numpy())

def build_model():
  model = keras.Sequential([
       layers.Dense(64, activation='relu', input_shape=[len(train_df.keys())]),
       layers.Dense(64, activation='relu'),
       layers.Dense(1)
       ])

  model.compile(loss='mean_squared_error',
                optimizer=keras.optimizers.Adam(learning_rate=0.001),
                metrics=['mae', 'mse'])
  return model

model = build_model()

In [ ]:
# Train model
history = model.fit(train_df, train_labels, epochs=1000, validation_split=0.2)


In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_df, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_df).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
